In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
from torchsummary import summary 
import onnx 
import onnxruntime as ort
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error
import random
import os
import datetime
import matplotlib.pyplot as plt

In [2]:
import os, sys
REPO = "/home/deepaksr/project/Project_files_2/U-2-Net"    
os.chdir(REPO)
sys.path.insert(0, REPO)

In [3]:
from model import U2NET, U2NETP

In [4]:
class Saliency_RGB_v1(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v1, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
class Saliency_RGB_v2(nn.Module):
    def __init__(self):
        super(Saliency_RGB_v2, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(8, 1, kernel_size=2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### d1

In [ ]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d1_20250511_201128/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d1_fixed_bce_20250514_081726/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d1_fixed_bce_20250514_090812/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(3,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Your Saliency_v1
model_v1 = Saliency_RGB_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Your Saliency_v2
model_v2 = Saliency_RGB_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3255485/1734783359.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_32554

In [ ]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/RGB/simulated_dataset"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_rgb_d1.pt")
print("Saved u2netp_finetuned_rgb_d1.pt")

# 2) Script Saliency_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_rgb_v1_d1.pt")
print("Saved saliency_rgb_v1_d1.pt")

# 3) Script Saliency_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_rgb_v2_d1.pt")
print("Saved saliency_rgb_v2_d1.pt")


Saved u2netp_finetuned_rgb_d1.pt
Saved saliency_rgb_v1_d1.pt
Saved saliency_rgb_v2_d1.pt


### d2

In [10]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d2_20250511_205923/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d2_fixed_bce_20250514_095620/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d2_fixed_bce_20250515_130907/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(3,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Your Saliency_v1
model_v1 = Saliency_RGB_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Your Saliency_v2
model_v2 = Saliency_RGB_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3255485/66445236.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_3255485

In [11]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/RGB/paper_dataset"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_rgb_d2.pt")
print("Saved u2netp_finetuned_rgb_d2.pt")

# 2) Script Saliency_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_rgb_v1_d2.pt")
print("Saved saliency_rgb_v1_d2.pt")

# 3) Script Saliency_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_rgb_v2_d2.pt")
print("Saved saliency_rgb_v2_d2.pt")


Saved u2netp_finetuned_rgb_d2.pt
Saved saliency_rgb_v1_d2.pt
Saved saliency_rgb_v2_d2.pt


### d3

In [ ]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d3_20250511_212906/best_model.pth"
pth_v1 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v1_d3_fixed_bce_20250517_153240/best_model.pth"
pth_v2 = "/home/deepaksr/project/Project_files_2/training_logs/RGB_v2_d3_fixed_bce_20250518_022557/best_model.pth"

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(3,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Your Saliency_v1
model_v1 = Saliency_RGB_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Your Saliency_v2
model_v2 = Saliency_RGB_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")


All models loaded to CPU and set to eval()


/tmp/ipykernel_3255485/686869804.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_u2.load_state_dict(torch.load(pth_u2, map_location=device))
/tmp/ipykernel_325548

In [9]:
# Cell: Script and save each model

# Make output directory
out_dir = "/home/deepaksr/project/Project_files_2/saliency_models_pt/RGB/military_dataset"
os.makedirs(out_dir, exist_ok=True)

# 1) Script U2NETP
scripted_u2 = torch.jit.script(model_u2)
scripted_u2.save(f"{out_dir}/u2netp_finetuned_rgb_d3.pt")
print("Saved u2netp_finetuned_rgb_d3.pt")

# 2) Script Saliency_v1
scripted_v1 = torch.jit.script(model_v1)
scripted_v1.save(f"{out_dir}/saliency_rgb_v1_d3.pt")
print("Saved saliency_rgb_v1_d3.pt")

# 3) Script Saliency_v2
scripted_v2 = torch.jit.script(model_v2)
scripted_v2.save(f"{out_dir}/saliency_rgb_v2_d3.pt")
print("Saved saliency_rgb_v2_d3.pt")


Saved u2netp_finetuned_rgb_d3.pt
Saved saliency_rgb_v1_d3.pt
Saved saliency_rgb_v2_d3.pt


## Combined

In [ ]:
# Paths to your .pth checkpoints
pth_u2 = "/home/deepaksr/project/Project_files_2/U-2-Net/training_logs/u2netp_finetune_rgb_d4_combined_20250517_080546/best_model.pth"
pth_v1 = ""
pth_v2 = ""

device = torch.device("cpu")   # Load to CPU for portability

# 1) U2NETP saliency
model_u2 = U2NETP(3,1).to(device).eval()
model_u2.load_state_dict(torch.load(pth_u2, map_location=device))

# 2) Your Saliency_v1
model_v1 = Saliency_RGB_v1().to(device).eval()
model_v1.load_state_dict(torch.load(pth_v1, map_location=device))

# 3) Your Saliency_v2
model_v2 = Saliency_RGB_v2().to(device).eval()
model_v2.load_state_dict(torch.load(pth_v2, map_location=device))

print("All models loaded to CPU and set to eval()")
